#**2. Instalación de Conda y programas**

Instale y llame a conda.

In [ ]:
pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda --version

Instale sus programas.

In [ ]:
!conda install -c conda-forge -c bioconda -c defaults abricate

In [ ]:
!abricate --check

In [ ]:
!conda install bioconda::flye

In [ ]:
%%shell
conda install -c bioconda quast -y

In [ ]:
!conda create -yqn tutorial -c conda-forge -c bioconda kraken2 krakentools bracken
conda_path = !source activate tutorial && echo $CONDA_PREFIX:$PATH
%set_env PATH=$conda_path

In [ ]:
!conda install -c bioconda krona -y

#**3. Ensamblaje (metaFlye)**

A continuación usted realizará el ensamblaje *de novo* del metagenoma de la muestra procesada. En este paso se utilizará el programa `flye` con habilitando la opción para metagenomas `--meta`.

**Nota:**
- Esto tarda aproximadamente 40 minutos para el archivo completo `MetaG.fastq`).
- El archivo `MetaG_50p.fastq` es subset de datos que corresponde al 50% de las lecturas y tarda solo 15 minutos.
- El archivo `MetaG_30p.fastq` es subset de datos que corresponde al 30% de las lecturas y tarda solo 5 minutos.

Descargue los archivos fastq.

In [ ]:
!wget -O MetaG_30p.fastq https://www.dropbox.com/scl/fi/z7ia4jbkpzb4pvwwiuz2v/MetaG_30p.fastq?rlkey=0eyuhu0lz7lp4hb2nt1xnab2y&st=znlx6st4&dl=0

In [ ]:
!wget -O MetaG_50p.fastq https://www.dropbox.com/scl/fi/pedbg1mrdgv1e7tlkhkfh/MetaG_50p.fastq?rlkey=m7m899j6rjtufb28jikpq620k&st=uv6vy5a3&dl=0

Observe las primeras lineas de su archivo.

In [ ]:
!head MetaG_50p.fastq

Ejecute flye.

In [ ]:
%%shell
flye --meta --nano-raw MetaG_50p.fastq --out-dir /content/assembly --threads 2

#**4. Evaluación de los genomas ensamblados.**

##**4.1. Clasificación taxonómica**

Primero que todo deberemos identificar taxonómicamente los genomas obtenidos. Podemos encontrar varias opciones, entre estas podemos hacer uso del programa `kraken2`.

###**4.1.2. Descargar base de datos**

Primero deberemos descargar la base de datos de refencia. La decisión de cual base de datos usar influenciará en la cantidad de contigs identificados, asi que sea cauteloso en esta decisión.

**Nota:** Este proceso tarda aproximadamente 8 minutos.

In [ ]:
!mkdir protocol_db
!wget -qnc https://genome-idx.s3.amazonaws.com/kraken/k2_standard_eupath_20201202.tar.gz
!tar xvfz k2_standard_eupath_20201202.tar.gz -C protocol_db

###**4.1.3. Clasifique los contigs con Kraken2**

A continuación realice la identificación taxonómica de los contigs, obtendrá un archivo de salida y un reporte.

In [ ]:
!kraken2 --db protocol_db --report metaG.k2report --minimum-hit-groups 3 /content/assembly/assembly.fasta > metaG.kraken2

Observe el número de contigs clasificados.

###**4.1.4. Estime las abundancias con Bracken**

In [ ]:
!bracken -d protocol_db -i metaG.k2report -o metaG.bracken -w metaG.breport -r 100 -l S -t 2

Observe el resultado de bracken.

In [ ]:
!head metaG.bracken

###**4.1.5. Visualice la taxonomía con Krona**

Cambie el formato de bracken a krona.

In [ ]:
!kreport2krona.py -r metaG.breport -o metaG.b.krona.txt --no-intermediate-ranks

Actualice la base de datos de taxonomía de krona.

In [ ]:
!ktUpdateTaxonomy.sh

Genere el reporte de krona.

In [ ]:
!ktImportTaxonomy -t 2 -m 6 metaG.bracken -o metaG.krona.out.html

Descargue el reporte y visualice el resultado directamente en algún navegador.

In [ ]:
from google.colab import files
files.download('metaG.krona.out.html')

In [ ]:
from google.colab import files
files.download('metaG.breport')

###**4.1.6. Visualice los resultados con Pavian**

Usted también podrá usar los archivos de reporte de bracken para explorar los resultados en la página web de Pavian.

In [ ]:
from IPython.display import IFrame
IFrame('https://fbreitwieser.shinyapps.io/pavian/', width='100%', height=600)

##**5.2. Estadísticas del ensamblaje**

Primero podemos descargar la base de datos de silva para quast. Nos será de utilidad más adelante.

In [ ]:
%%bash
quast-download-silva

Observe las opciones de `quast` desplegando el menú de help.

In [ ]:
!metaquast.py -h

Corra `metaquast` especificando los genomas de refencia que desea analizar desde el archivo `list_references.txt` que se encuentra en la carpeta `data` dentro de `Modulo_4`. Usted puede crear este archivo usando el resultado obtenido en krake2/bracken.

**Nota:** Esto tarda aproximadamente 12 minutos para el 100%
y 4 minutos para 50%.

Descargue el archivo con la lista de especies.

In [ ]:
!wget -O list_references_50p.txt https://www.dropbox.com/scl/fi/m8ow3goeyfvx6it16pln4/list_references_50p.txt?rlkey=fv93p4jfgjzfovmmtab9r62oi&st=j4c9hbl7&dl=0

Ejecute metaquast.

In [ ]:
%%shell
metaquast.py /content/assembly/assembly.fasta -t 2 --references-list list_references_50p.txt

Otra opción es ejecutar este comando pero en lugar de especificar una lista de genomas usted puede usar la opción `--gene-finding`, el cual contrastará los contigs contra la base de datos de silva que previamente descargamos para identificar los genomas de referencia a descargar.

**Nota:** Esto tarda aproximadamente 15 minutos.

**Nota2:** No haremos esta opción por eso esta como un comentario (#), pero si desea puede hacerlo y comparar los resultados de ambas opciones.

In [ ]:
%%shell
#metaquast.py /content/assembly/assembly.fasta -t 2 --gene-finding

In [ ]:
%ls /content/quast_results/latest/

In [ ]:
import IPython
IPython.display.HTML(filename='/content/quast_results/latest/report.html')

In [ ]:
import IPython
IPython.display.HTML(filename='/content/quast_results/latest/icarus.html')

# **5. ABRicate. Resistencia a antibióticos**

Identifica genes de resistencia a antibióticos mediante la ejecución de ensamblajes locales.

Cribado masivo de contigs para genes de resistencia a los antimicrobianos o de virulencia. Incluye varias bases de datos: `NCBI`, `CARD`, `ARG-ANNOT`, `Resfinder`, `MEGARES`, `EcOH`, `PlasmidFinder`, `Ecoli_VF` y `VFDB`.

¿Es esta la herramienta adecuada para mí?
- Solo admite contigs, no lecturas FASTQ
- Solo detecta genes de resistencia adquiridos, **NO** mutaciones puntuales
- Utiliza una base de datos de secuencias de ADN, no de proteínas.
- Necesita BLAST+ >= 2.7 y any2fastaestar instalado
- Está escrito en Perl 🐫

Antes de ejecutar el programa confirme si funciona correctamente y que cumple con todos los requisitos.

In [ ]:
!abricate --check

El error puede ser ocasionado por el cambio del enviroment cuando instalamos `kraken2`. Vuelva a colocarlo en base.

In [ ]:
!echo $PATH

In [ ]:
%set_env PATH=['/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin' ]

In [ ]:
!echo $PATH

Observe las bases de datos disponibles para ABRicate.

In [ ]:
!abricate --list

Usted puede actualizar cada una de las bases de datos disponibles con el comando `abricate-get_db`.

In [ ]:
!abricate-get_db --help

In [ ]:
!abricate-get_db --db card --force

Observe la actualización de los registros de la base de datos.

In [ ]:
!abricate --list

Abricate acepta cualquier archivo de secuencia que any2fasta pueda convertirse a archivos FASTA (por ejemplo, Genbank, EMBL) y pueden ser opcionalmente gzipcomprimidos bzip2.

**Nota:** No acepta lecturas FASTQ sin procesar; utilice otros programas como Ariba o SRTS2 para eso.



In [ ]:
!abricate /content/assembly/assembly.fasta --db ncbi > abricate_output_ncbi.tsv

Los resultados son almacenados en una archivo `.tsv`.

In [ ]:
!cat abricate_output_ncbi.tsv

Podemos probar otras bases de datos. Sientase libre de probarlas.

In [ ]:
!abricate /content/assembly/assembly.fasta --db megares > abricate_output_megares.tsv

In [ ]:
!cat abricate_output_megares.tsv

**Advertencias**

- No encuentra resistencia mutacional, sólo genes adquiridos.
- Informe de brechas incompleto
- A veces se informarán dos genes muy superpuestos para el mismo locus.
- Posibles problemas en el cálculo de la cobertura.

La base de datos predeterminada es ncbi. Puede elegir una base de datos diferente mediante la opción `--db`.

Realice el análisis cambiado la base de datos a vcfdb.


In [ ]:
!abricate --db vfdb /content/assembly/assembly.fasta > abricate_output_vfdb.tsv

Observe el resultado con vfdb.

In [ ]:
!cat abricate_output_vfdb.tsv

Ahora con Card.

In [ ]:
!abricate --db card /content/assembly/assembly.fasta > abricate_output_card.tsv

Observe el resultado con Card.

In [ ]:
!cat abricate_output_card.tsv

Si usted desea comparar varias muestras use el comando `summary` y enliste todos los output de las distintas muestras. Para ejemplificar creemos una copia de nuestro resultado, o realice los pasos anteriores para otra muestra.



Creemos la copia.

In [ ]:
!cp abricate_output_ncbi.tsv abricate_output2_ncbi.tsv

Comparemos los resultados.

In [ ]:
!abricate --summary abricate_output_ncbi.tsv abricate_output2_ncbi.tsv > tabla_resultado_ncbi.tsv

In [ ]:
!cat tabla_resultado_ncbi.tsv